# Reading and Writing to GCS Using the Storage Editor Service Account
Read the docs at:<br>
https://filesystem-spec.readthedocs.io/en/latest/index.html<br>
https://gcsfs.readthedocs.io/en/latest/

In [1]:
import numpy as np
import xarray as xr
xr.set_options(display_style="html")
import gcsfs
import fsspec

## Write an Xarray dataset

### Initialize fsspec mapping object
More info: https://filesystem-spec.readthedocs.io/en/latest/api.html#fsspec.get_mapper

In [2]:
outfile = fsspec.get_mapper('gs://ldeo-glaciology-test/asdf.zarr', mode='ab',
                            token='../secrets/ldeo-glaciology-bc97b12df06b.json')
type(outfile)

fsspec.mapping.FSMap

In [3]:
outfile?

Type:        FSMap
String form: <fsspec.mapping.FSMap object at 0x7f075b26aa50>
Length:      0
File:        /srv/conda/envs/notebook/lib/python3.7/site-packages/fsspec/mapping.py
Docstring:  
Wrap a FileSystem instance as a mutable wrapping.

The keys of the mapping become files under the given root, and the
values (which must be bytes) the contents of those files.

Parameters
----------
root: string
    prefix for all the files
fs: FileSystem instance
check: bool (=True)
    performs a touch at the location, to check for write access.

Examples
--------
>>> fs = FileSystem(**parameters) # doctest: +SKIP
>>> d = FSMap('my-data/path/', fs) # doctest: +SKIP
or, more likely
>>> d = fs.get_mapper('my-data/path/')

>>> d['loc1'] = b'Hello World' # doctest: +SKIP
>>> list(d.keys()) # doctest: +SKIP
['loc1']
>>> d['loc1'] # doctest: +SKIP
b'Hello World'


### Create Xarray dataset

In [4]:
da = xr.DataArray(np.random.randn(2, 3), dims=('x', 'y'), coords={'x': [10, 20]})
da

<xarray.DataArray (x: 2, y: 3)>
array([[ 0.10163346,  1.23833892,  0.33041276],
       [-0.61834809, -1.05297875,  1.20129227]])
Coordinates:
  * x        (x) int64 10 20
Dimensions without coordinates: y

In [5]:
ds = da.to_dataset(name='asdf')
ds

<xarray.Dataset>
Dimensions:  (x: 2, y: 3)
Coordinates:
  * x        (x) int64 10 20
Dimensions without coordinates: y
Data variables:
    asdf     (x, y) float64 0.1016 1.238 0.3304 -0.6183 -1.053 1.201

### Write Xarray dataset to the ldeo-glaciology-test bucket
More info: http://xarray.pydata.org/en/stable/generated/xarray.Dataset.to_zarr.html

In [6]:
ds.to_zarr(outfile, mode='w');

## List files in the ldeo-glaciology-test bucket

In [8]:
fs = gcsfs.GCSFileSystem(project='ldeo-glaciology', mode='ab',
                         token='../secrets/ldeo-glaciology-bc97b12df06b.json',
                        cache_timeout = 0)

In [9]:
fs.ls('ldeo-glaciology-test')

['ldeo-glaciology-test/asdf.zarr']

In [10]:
fs.listdir('ldeo-glaciology-test/asdf.zarr')

[{'kind': 'storage#object',
  'id': 'ldeo-glaciology-test/asdf.zarr/.zattrs/1594929505772721',
  'selfLink': 'https://www.googleapis.com/storage/v1/b/ldeo-glaciology-test/o/asdf.zarr%2F.zattrs',
  'mediaLink': 'https://www.googleapis.com/download/storage/v1/b/ldeo-glaciology-test/o/asdf.zarr%2F.zattrs?generation=1594929505772721&alt=media',
  'name': 'ldeo-glaciology-test/asdf.zarr/.zattrs',
  'bucket': 'ldeo-glaciology-test',
  'generation': '1594929505772721',
  'metageneration': '1',
  'contentType': 'application/octet-stream',
  'storageClass': 'STANDARD',
  'size': 2,
  'md5Hash': 'mZFLkyvTelC5g8XnyQrpOw==',
  'crc32c': 'KXvQqg==',
  'etag': 'CLHJiLXH0uoCEAE=',
  'timeCreated': '2020-07-16T19:58:25.772Z',
  'updated': '2020-07-16T19:58:25.772Z',
  'timeStorageClassUpdated': '2020-07-16T19:58:25.772Z',
  'type': 'file'},
 {'kind': 'storage#object',
  'id': 'ldeo-glaciology-test/asdf.zarr/.zgroup/1594929504645520',
  'selfLink': 'https://www.googleapis.com/storage/v1/b/ldeo-glaciolo

## Write text to a new file in a new folder in a bucket

In [11]:
openfile = fsspec.open('gs://ldeo-glaciology-test/mynewdir/asdf.txt', mode='ab', token='../secrets/ldeo-glaciology-bc97b12df06b.json')

In [12]:
with openfile as f:
    f.write(b'This is some really great prose.')

In [13]:
fs.ls('ldeo-glaciology-test')

['ldeo-glaciology-test/asdf.zarr']

In [14]:
fs.invalidate_cache()

In [15]:
fs.ls('ldeo-glaciology-test')

['ldeo-glaciology-test/asdf.zarr', 'ldeo-glaciology-test/mynewdir']

The question mark is a great way to learn more about objects:

In [16]:
openfile?

Type:        OpenFile
String form: <OpenFile 'ldeo-glaciology-test/mynewdir/asdf.txt'>
File:        /srv/conda/envs/notebook/lib/python3.7/site-packages/fsspec/core.py
Docstring:  
File-like object to be used in a context

Can layer (buffered) text-mode and compression over any file-system, which
are typically binary-only.

These instances are safe to serialize, as the low-level file object
is not created until invoked using `with`.

Parameters
----------
fs: FileSystem
    The file system to use for opening the file. Should match the interface
    of ``dask.bytes.local.LocalFileSystem``.
path: str
    Location to open
mode: str like 'rb', optional
    Mode of the opened file
compression: str or None, optional
    Compression to apply
encoding: str or None, optional
    The encoding to use if opened in text mode.
errors: str or None, optional
    How to handle encoding errors if opened in text mode.
newline: None or str
    Passed to TextIOWrapper in text mode, how to handle line endin

## Upload a local file to a folder in a bucket

In [17]:
with open('myawesomefile.txt', 'wb') as f:
    f.write(b'This is some really really great prose.')

In [18]:
fs.put?

Signature: fs.put(lpath, rpath, recursive=False, **kwargs)
Docstring: Upload file from local 
File:      /srv/conda/envs/notebook/lib/python3.7/site-packages/fsspec/spec.py
Type:      method


In [19]:
fs.put('myawesomefile.txt', 'gs://ldeo-glaciology-test/myawesomedir/asdf.txt')

In [20]:
fs.ls('ldeo-glaciology-test')

['ldeo-glaciology-test/asdf.zarr',
 'ldeo-glaciology-test/myawesomedir',
 'ldeo-glaciology-test/mynewdir']

In [21]:
fs.ls('ldeo-glaciology-test/myawesomedir/')

['ldeo-glaciology-test/myawesomedir/asdf.txt']